In [9]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#   Modelos lineales y modelos lineales generalizados
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Julio 2024
# ==========================================================
# Modelo lineal multivariante: evaluacion de modelos 
# Linealidad, normalidad, homocedasticidad y correlacion de los errores 

# Importando librerias:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms


# Cargar el conjunto de datos:
url = 'https://raw.githubusercontent.com/rogon666/UMSA/main/MLMLG/datos/salarios.csv'

# Cargar los datos en un DataFrame
datos = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(datos.head())

   salario  educacion  educacionmadre  edad  antiguedad_ejecutivo  \
0     1161         12              11    49                     6   
1      600         10               9    43                     5   
2      379          9               8    51                     5   
3      651          8               9    55                     4   
4      497         10               9    44                     5   

   educacion_grado  antiguedad  educacion_posgrado  valor_empresa  \
0                1           9                   1          23200   
1                1          10                   1           1100   
2                1           9                   1           1100   
3                1          22                   0           1000   
4                1           8                   1            387   

   beneficios_empresa  ventas_empresa  
0                 966            6200  
1                  48             283  
2                  40             169  
3         

In [10]:
# Definir las variables independientes y dependientes
X = datos[['educacion', 'edad', 'educacion_posgrado']]
y = datos['salario']

# Añadir una constante a las variables independientes
X = sm.add_constant(X)

# Ajustar el modelo de regresión lineal
modelo_OLS = sm.OLS(y, X).fit()

# Resumen del modelo
print(modelo_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                salario   R-squared:                       0.929
Model:                            OLS   Adj. R-squared:                  0.928
Method:                 Least Squares   F-statistic:                     754.5
Date:                Wed, 24 Jul 2024   Prob (F-statistic):           4.39e-99
Time:                        21:58:44   Log-Likelihood:                -1145.1
No. Observations:                 177   AIC:                             2298.
Df Residuals:                     173   BIC:                             2311.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -1920.2137     97

In [11]:
# Ajustando el modelo de regresion utilizando una formula:
resultados = smf.ols("salario ~ educacion + edad + educacion_posgrado", data=datos).fit()

# Resultados
print(resultados.summary())

                            OLS Regression Results                            
Dep. Variable:                salario   R-squared:                       0.929
Model:                            OLS   Adj. R-squared:                  0.928
Method:                 Least Squares   F-statistic:                     754.5
Date:                Wed, 24 Jul 2024   Prob (F-statistic):           4.39e-99
Time:                        21:58:47   Log-Likelihood:                -1145.1
No. Observations:                 177   AIC:                             2298.
Df Residuals:                     173   BIC:                             2311.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept          -1920.2137     97

In [12]:
# Test de linealidad: Harvey-Collier
test = sms.linear_harvey_collier(resultados)
print(f'Estadígrafo HC: {test[0]}')
print(f'p-value HC ~ t(n-k-1): {test[1]}')

Estadígrafo HC: 0.5802422316806765
p-value HC ~ t(n-k-1): 0.5625062662422351


In [13]:
# Test de normalidad de residuos: Jarque-Bera
test = sms.jarque_bera(resultados.resid)
print(f'Estadígrafo Jarque-Bera (JB): {test[0]}')
print(f'p-value JB ~ chi^2(2): {test[1]}')
print(f'Sesgo: {test[2]}')
print(f'Curtosis: {test[3]}')

Estadígrafo Jarque-Bera (JB): 21651.867549974988
p-value JB ~ chi^2(2): 0.0
Sesgo: 5.504584983234719
Curtosis: 56.053224445867784


In [14]:
# Test de normalidad de residuos: Omnibus de D’Agostino and Pearson
test = sms.omni_normtest(resultados.resid)
print(f'Estadígrafo Omni: {test[0]}')
print(f'p-value omni ~ chi^2(2): {test[1]}')
# D’Agostino, R. B. (1971), “An omnibus test of normality for moderate and large sample size”, Biometrika, 58, 341-348
# D’Agostino, R. and Pearson, E. S. (1973), “Tests for departure from normality”, Biometrika, 60, 613-622

Estadígrafo Omni: 240.65557645872698
p-value omni ~ chi^2(2): 5.524659957306687e-53


In [15]:
# Test Homocedasticidad: Breusch–Pagan
test = sms.het_breuschpagan(resultados.resid, resultados.model.exog)
print(f'Estadígrafo LMBP: {test[0]}')
print(f'p-value LMBP ~ chi^2(k-1): {test[1]}')
print(f'Estadígrafo FBP: {test[2]}')
print(f'p-value FBP ~ F(k,n-k-1): {test[3]}')

Estadígrafo LMBP: 28.58445881065216
p-value LMBP ~ chi^2(k-1): 2.7380083970109536e-06
Estadígrafo FBP: 11.106454518654198
p-value FBP ~ F(k,n-k-1): 1.0533061229143745e-06


In [16]:
# Test Homocedasticidad: Goldfeld-Quandt
test = sms.het_goldfeldquandt(resultados.resid, resultados.model.exog)
print(f'Estadígrafo FGQ: {test[0]}')
print(f'p-value FGQ ~ F(n1-k,n2-k): {test[1]}')

Estadígrafo FGQ: 5.020218865306849
p-value FGQ ~ F(n1-k,n2-k): 8.694729000659282e-13


In [17]:
# Test Homocedasticidad: White
from statsmodels.stats.diagnostic import het_white
white_test = het_white(resultados.resid, resultados.model.exog)
print(f'Estadístico LMW: {white_test[0]}')
print(f'p-value LMW ~ chi^2(k-1): {white_test[1]}')

Estadístico LMW: 127.49891358864983
p-value LMW ~ chi^2(k-1): 9.328969637031958e-24


In [19]:
# Transformaciones de variables
resultados_logsal = smf.ols("np.log(salario) ~ educacion + edad + educacion_posgrado", data=datos).fit()

# Resultados
print(resultados_logsal.summary())

# Test Homocedasticidad: White
from statsmodels.stats.diagnostic import het_white
white_test = het_white(resultados_logsal.resid, resultados_logsal.model.exog)
print(f'Estadístico LMW: {white_test[0]}')
print(f'p-value LMW ~ chi^2(k-1): {white_test[1]}')

                            OLS Regression Results                            
Dep. Variable:        np.log(salario)   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     300.3
Date:                Wed, 24 Jul 2024   Prob (F-statistic):           2.52e-68
Time:                        22:02:33   Log-Likelihood:               -0.44019
No. Observations:                 177   AIC:                             8.880
Df Residuals:                     173   BIC:                             21.58
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              3.9092      0

In [ ]:
# Calcular errores estándar robustos usando la matriz de covarianza de tipo HC3
robust_cov = modelo_OLS.get_robustcov_results(cov_type='HC3')

# Mostrar los resultados con errores estándar robustos
print(robust_cov.summary())

In [ ]:
# Estimacion con minimos cuadrados ponderados (WLS):
import matplotlib.pyplot as plt

# Calcular los residuos del modelo OLS
residuos = modelo_OLS.resid

# Calcular los pesos inversamente proporcionales a los residuos al cuadrado (o a otra medida de varianza)
pesos = 1 / (residuos**2)

# Ajustar el modelo de regresión ponderada usando WLS
wls_model = sm.WLS(y, X, weights=pesos).fit()

# Mostrar el resumen del modelo OLS
print("Resumen del modelo OLS:")
print(modelo_OLS.summary())

# Mostrar el resumen del modelo WLS
print("\nResumen del modelo WLS:")
print(wls_model.summary())

# Graficar los residuos para observar la heterocedasticidad
plt.scatter(modelo_OLS.fittedvalues, residuos)
plt.xlabel('Valores Ajustados')
plt.ylabel('Residuos')
plt.title('Gráfico de Residuos (OLS)')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

# Graficar los residuos ponderados para observar la heterocedasticidad corregida
plt.scatter(wls_model.fittedvalues, wls_model.resid)
plt.xlabel('Valores Ajustados')
plt.ylabel('Residuos Ponderados')
plt.title('Gráfico de Residuos Ponderados (WLS)')
plt.axhline(y=0, color='r', linestyle='--')
plt.show()

In [ ]:
# Correlacion de los errores: Test de Durbin-Watson
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import norm
dw_stat = durbin_watson(modelo_OLS.resid)
print(f'Estadística de Durbin-Watson: {dw_stat}')
# Interpretación basada en valores críticos (asumiendo un nivel de significancia de 0.05)
# Nota: Los valores críticos dL y dU dependen de n y k
dL = 1.643 # Valor crítico inferior para n = 177 y k = 3 (excluyendo la constante)
dU = 1.704  # Valor crítico superior para n = 177 y k = 3 (excluyendo la constante)
if dw_stat < dL:
    print("Evidencia de autocorrelación positiva.")
elif dL <= dw_stat <= dU:
    print("Prueba inconclusa.")
else:
    print("No hay evidencia de autocorrelación positiva.")
# Nota: el estadigrafo DW es sesgado (subestima autocorrelacion) en el contexto de AR(I)MA
# Calcular la estadística H de Durbin-Watson
residuos = modelo_OLS.resid
rho_hat = np.corrcoef(residuos[:-1], residuos[1:])[0, 1]
n = len(modelo_OLS.resid)
k = 5  # Número de variables explicativas excluyendo la constante
h_stat = (1 - dw_stat / 2) * np.sqrt(n / (1 - n * (rho_hat**2)))
print(f'Estadígrafo H de Durbin-Watson: {h_stat}')
# Evaluar la estadística H usando la distribución normal estándar
alpha = 0.05
z_critical = norm.ppf(1 - alpha / 2)  # Valor crítico para un nivel de significancia de 5%
print(f'Valor crítico z para H: {z_critical}')

In [ ]:
# Correlacion de los errores: Test Breusch-Godfrey
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
bg_test = acorr_breusch_godfrey(modelo_OLS, nlags=5)

# Resultados del test de Breusch-Godfrey
print(f'Estadístico LM: {bg_test[0]}')
print(f'Valor p (LM): {bg_test[1]}')
print(f'Estadístico F: {bg_test[2]}')
print(f'Valor p (F): {bg_test[3]}')

# Interpretación de los resultados
alpha = 0.05
if bg_test[1] < alpha:
    print("Se rechaza la hipótesis nula de no correlación de los errores. \nExiste evidencia de correlación de los errores.")
else:
    print("No se rechaza la hipótesis nula de no correlación de los errores. \nNo hay evidencia de correlación de los errores.")